<a href="https://colab.research.google.com/github/nathUjjal/Multi_Modal_Fake_News_Detection/blob/main/TIYASHA_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# verification.py

from sentence_transformers import SentenceTransformer, util
import torch

# Load the pre-trained model. This is done once when the script is loaded
# to avoid reloading the model on every function call, which is inefficient.
print("Loading sentence-transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded successfully.")

def verify_claim(claim: str, evidences: list[str]):
    """
    Verifies a claim against a list of evidence snippets using semantic similarity.

    Args:
        claim (str): The statement to be verified.
        evidences (list[str]): A list of evidence strings to check against the claim.

    Returns:
        dict: A dictionary containing the 'verdict' and the 'confidence_score'.
    """
    if not evidences:
        return {
            'verdict': '⚠️ Uncertain',
            'confidence_score': 0.0,
            'reason': 'No evidence provided.'
        }

    # 1. Encode the claim and all evidence snippets into embeddings
    claim_embedding = model.encode(claim, convert_to_tensor=True)
    evidence_embeddings = model.encode(evidences, convert_to_tensor=True)

    # 2. Compute cosine similarity between the claim and each evidence snippet
    # The result is a tensor of similarity scores.
    cosine_scores = util.pytorch_cos_sim(claim_embedding, evidence_embeddings)

    # 3. Aggregate the scores. We take the highest score as the most relevant
    # piece of evidence is what matters most.
    # The .item() method extracts the scalar value from the tensor.
    confidence_score = torch.max(cosine_scores).item()

    # 4. Determine the final verdict based on the aggregated score
    if confidence_score > 0.75:
        verdict = '✅ Likely True'
    elif confidence_score < 0.4:
        verdict = '❌ Likely Fake'
    else:
        verdict = '⚠️ Uncertain'

    return {
        'verdict': verdict,
        'confidence_score': round(confidence_score, 4)
    }

# This block demonstrates how to use the function and will only run
# when the script is executed directly (e.g., `python verification.py`)
if __name__ == "__main__":
    # --- Example 1: Clear Support ---
    claim1 = "Gemini is a large language model created by Google."
    evidences1 = [
        "Google's latest AI is a multimodal model called Gemini.",
        "The Eiffel Tower is in Paris, France.",
        "According to official sources, Google developed the Gemini family of models."
    ]
    result1 = verify_claim(claim1, evidences1)
    print(f"\n--- Verifying Claim 1 ---")
    print(f"Claim: '{claim1}'")
    print(f"Result: {result1['verdict']} (Confidence: {result1['confidence_score']})")

    # --- Example 2: Clear Contradiction (or lack of support) ---
    claim2 = "The capital of Australia is Sydney."
    evidences2 = [
        "Canberra was designated as the capital of Australia in 1908.",
        "Sydney is the most populous city in Australia.",
        "Australia is a continent and a country."
    ]
    result2 = verify_claim(claim2, evidences2)
    print(f"\n--- Verifying Claim 2 ---")
    print(f"Claim: '{claim2}'")
    print(f"Result: {result2['verdict']} (Confidence: {result2['confidence_score']})")


    # --- Example 3: Ambiguous/Uncertain ---
    claim3 = "This new diet plan is effective for everyone."
    evidences3 = [
        "Some participants in a study reported weight loss on the diet plan.",
        "Dietary needs vary significantly from person to person.",
        "Results from diet plans can be influenced by many factors like genetics and lifestyle."
    ]
    result3 = verify_claim(claim3, evidences3)
    print(f"\n--- Verifying Claim 3 ---")
    print(f"Claim: '{claim3}'")
    print(f"Result: {result3['verdict']} (Confidence: {result3['confidence_score']})")

Loading sentence-transformer model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully.

--- Verifying Claim 1 ---
Claim: 'Gemini is a large language model created by Google.'
Result: ⚠️ Uncertain (Confidence: 0.728)

--- Verifying Claim 2 ---
Claim: 'The capital of Australia is Sydney.'
Result: ✅ Likely True (Confidence: 0.7796)

--- Verifying Claim 3 ---
Claim: 'This new diet plan is effective for everyone.'
Result: ⚠️ Uncertain (Confidence: 0.5871)


In [ ]:
# Define some dummy data for testing
test_claim = "The sun is a star."
test_evidences = [
    "The sun is the star at the center of the Solar System.",
    "Planets orbit stars.",
    "The moon is a natural satellite of the Earth."
]

# You can then use this dummy data with your verify_claim function
# For example:
# result_test = verify_claim(test_claim, test_evidences)
# print(f"\n--- Verifying Test Claim ---")
# print(f"Claim: '{test_claim}'")
# print(f"Result: {result_test['verdict']} (Confidence: {result_test['confidence_score']})")

print("Dummy data created:")
print(f"Claim: {test_claim}")
print(f"Evidences: {test_evidences}")

Dummy data created:
Claim: The sun is a star.
Evidences: ['The sun is the star at the center of the Solar System.', 'Planets orbit stars.', 'The moon is a natural satellite of the Earth.']
